In [1]:
import json
token = {"username":"","key":""}
username = input()
key = input()
with open('/content/kaggle.json','w') as f:
  json.dump(token, f)

In [2]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v /content     # /content可替换成自己云盘里你想存放数据集的地址

- path is now set to: /content


In [3]:
!kaggle datasets download -d tongpython/cat-and-dog

 94% 205M/218M [00:02<00:00, 95.5MB/s]
100% 218M/218M [00:02<00:00, 88.9MB/s]


In [ ]:
!unzip /content/datasets/tongpython/cat-and-dog/cat-and-dog.zip

In [4]:
cats_filename = '/content/training_set/training_set/cats'
dogs_filename = '/content/training_set/training_set/dogs'
savePath = '/content/ProccessedData'

In [24]:
class dataLoader():
  def __init__(self, cats_filename, dogs_filename, savePath):
    self.cats_filename = cats_filename
    self.dogs_filename = dogs_filename
    self.cat_files = os.listdir(cats_filename)
    self.dog_files = os.listdir(dogs_filename)
    self.trainX = None
    self.trainY = None
    self.testX = None
    self.testY = None
    self.savePath = savePath
  def ProcessImg(self):
    tempX = []
    tempY = []
    idx = 0
    for file in self.cat_files:
      img = cv2.imread(f'{self.cats_filename}/{file}')
      if img is None:
        print(idx)
      else:
        img = cv2.resize(img, (224, 224))
        tempX.append(img)
        tempY.append(1)
        cv2.imwrite(f'{self.savePath}/cat_{idx}.jpg', img)
      idx += 1
    idx = 0
    for file in self.dog_files:
      img = cv2.imread(f'{self.dogs_filename}/{file}')
      if img is None:
        print(idx)
      else:
        img = cv2.resize(img, (224, 224))
        tempX.append(img)
        tempY.append(0)
        cv2.imwrite(f'{self.savePath}/dog_{idx}.jpg', img)
      idx += 1
    tempX = np.array(tempX)
    tempY = np.array(tempY)
    self.trainX, self.testX, self.trainY, self.testY = train_test_split(tempX, tempY, shuffle=True, train_size=0.95)
    self.trainY = to_categorical(self.trainY, 2)
    self.testY = to_categorical(self.testY, 2)


In [31]:
test_cats_filename = '/content/training_set/training_set/cats'
test_dogs_filename = '/content/training_set/training_set/dogs'
test_savePath = '/content/ProccessedData'

In [32]:
TestData = TestdataLoader(test_cats_filename, test_dogs_filename, test_savePath)

In [33]:
TestData.ProcessImg()

36
28


In [30]:
class TestdataLoader():
  def __init__(self, cats_filename, dogs_filename, savePath):
    self.cats_filename = cats_filename
    self.dogs_filename = dogs_filename
    self.cat_files = os.listdir(cats_filename)
    self.dog_files = os.listdir(dogs_filename)
    self.testX = None
    self.testY = None
    self.savePath = savePath
  def ProcessImg(self):
    tempX = []
    tempY = []
    idx = 0
    for file in self.cat_files:
      img = cv2.imread(f'{self.cats_filename}/{file}')
      if img is None:
        print(idx)
      else:
        img = cv2.resize(img, (224, 224))
        tempX.append(img)
        tempY.append(1)
        cv2.imwrite(f'{self.savePath}/cat_{idx}.jpg', img)
      idx += 1
    idx = 0
    for file in self.dog_files:
      img = cv2.imread(f'{self.dogs_filename}/{file}')
      if img is None:
        print(idx)
      else:
        img = cv2.resize(img, (224, 224))
        tempX.append(img)
        tempY.append(0)
        cv2.imwrite(f'{self.savePath}/dog_{idx}.jpg', img)
      idx += 1
    tempX = np.array(tempX)
    tempY = np.array(tempY)
    self.testX = tempX
    self.testY = tempY
    self.testY = to_categorical(self.testY, 2)


In [9]:
data = dataLoader(cats_filename, dogs_filename, savePath)

In [18]:
data.ProcessImg()

36
28


In [7]:
import cv2
import os

In [8]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split

In [11]:
def createModel():
    model = Sequential()
    model.add(Conv2D(
        input_shape=(224, 224, 3),
        filters=32,
        strides=1,
        kernel_size=3,
        activation='relu',
        kernel_initializer='VarianceScaling'
    ))

    model.add(MaxPool2D(
        pool_size=2,
        strides=2
    ))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())

    model.add(Conv2D(
        filters=64,
        strides=1,
        kernel_size=3,
        activation='relu',
        kernel_initializer='VarianceScaling'
    ))

    model.add(MaxPool2D(
        pool_size=2,
        strides=2
    ))

    model.add(BatchNormalization())


    model.add(Conv2D(
        filters=128,
        strides=1,
        kernel_size=3,
        activation='relu',
        kernel_initializer='VarianceScaling'
    ))

    model.add(MaxPool2D(
        pool_size=2,
        strides=2
    ))

    model.add(BatchNormalization())


    model.add(Conv2D(
        filters=128,
        strides=1,
        kernel_size=3,
        activation='relu',
        kernel_initializer='VarianceScaling'
    ))
    model.add(Dropout(0.5))
    model.add(MaxPool2D(
        pool_size=2,
        strides=2
    ))

    model.add(BatchNormalization())


    model.add(Flatten())

    model.add(Dense(
        units=512,
        kernel_initializer='VarianceScaling',
        activation='relu'
    ))
    model.add(Dense(
        units=2,
        kernel_initializer='VarianceScaling',
        activation='softmax'
    ))
    model.compile(optimizer=Adam(), metrics=['acc'], loss=BinaryCrossentropy())
    return model

In [19]:
model = createModel()
batch_size = 256
epochs = 10
model.fit(data.trainX, data.trainY, batch_size=batch_size, epochs = epochs, shuffle=True)
loss, accuracy = model.evaluate(data.testX, data.testY)

Epoch 1/10
29/29 [==============================] - 7s 206ms/step - loss: 1.5228 - acc: 0.5696
Epoch 2/10
29/29 [==============================] - 6s 209ms/step - loss: 0.7050 - acc: 0.6953
Epoch 3/10
29/29 [==============================] - 6s 208ms/step - loss: 0.5635 - acc: 0.7563
Epoch 4/10
29/29 [==============================] - 6s 207ms/step - loss: 0.4465 - acc: 0.8045
Epoch 5/10
29/29 [==============================] - 6s 207ms/step - loss: 0.3546 - acc: 0.8548
Epoch 6/10
29/29 [==============================] - 6s 207ms/step - loss: 0.3082 - acc: 0.8672
Epoch 7/10
29/29 [==============================] - 6s 207ms/step - loss: 0.2395 - acc: 0.9046
Epoch 8/10
29/29 [==============================] - 6s 208ms/step - loss: 0.1941 - acc: 0.9258
Epoch 9/10
29/29 [==============================] - 6s 206ms/step - loss: 0.1645 - acc: 0.9343
Epoch 10/10
26/26 [==============================] - 0s 10ms/step - loss: 1.5109 - acc: 0.6629


In [22]:
len(data.trainY)

7204

In [28]:
trainX, testX, trainY, testY = train_test_split(data.testX, data.testY, shuffle=True, train_size=0.8)

In [29]:
batch_size = 128
model.fit(trainX, trainY, batch_size=batch_size, epochs = epochs, shuffle=True)
loss, accuracy = model.evaluate(testX, testY)

Epoch 1/10
5/5 [==============================] - 1s 109ms/step - loss: 1.0785 - acc: 0.7563
Epoch 2/10
5/5 [==============================] - 1s 111ms/step - loss: 0.1996 - acc: 0.9187
Epoch 3/10
5/5 [==============================] - 1s 109ms/step - loss: 0.0933 - acc: 0.9750
Epoch 4/10
5/5 [==============================] - 1s 108ms/step - loss: 0.0463 - acc: 0.9937
Epoch 5/10
5/5 [==============================] - 1s 108ms/step - loss: 0.0230 - acc: 0.9969
Epoch 6/10
5/5 [==============================] - 1s 110ms/step - loss: 0.0085 - acc: 1.0000
Epoch 7/10
5/5 [==============================] - 1s 107ms/step - loss: 0.0072 - acc: 1.0000
Epoch 8/10
5/5 [==============================] - 1s 107ms/step - loss: 0.0044 - acc: 1.0000
Epoch 9/10
5/5 [==============================] - 1s 108ms/step - loss: 0.0022 - acc: 1.0000
Epoch 10/10
6/6 [==============================] - 0s 9ms/step - loss: 1.1500 - acc: 0.7516


In [34]:
loss, accuracy = model.evaluate(TestData.testX, TestData.testY)

251/251 [==============================] - 3s 10ms/step - loss: 1.0931 - acc: 0.7780


In [ ]:
model.save_weights('MyModel_Weights.h5')
import tensorflowjs
tensorflowjs.converters.save_keras_model(model, 'modelWeights')